In [12]:
import pulp
from datetime import datetime

# Example data
V = [1, 2]  # Vehicles
C = [1, 2, 3]  # Customers
P = [1, 2]  # Products
w = {1: 10, 2: 20}  # Weight of products
n = {(1, 1): 2, (1, 2): 1, (2, 1): 1, (2, 2): 1, (3, 1): 3, (3, 2): 2}  # Demand
TW = {1: [(20240608, 8, 15), (20240610, 9, 18)], 2: [(20240608, 10, 16)], 3: [(20240609, 11, 14)]}  # Time windows
D = {(0, 1): 10, (0, 2): 20, (0, 3): 30, (1, 2): 15, (1, 3): 25, (2, 3): 35, (1, 0): 10, (2, 0): 20, (3, 0): 30, (2, 1): 15, (3, 1): 25, (3, 2): 35}  # Distances
T = {(0, 1): 1, (0, 2): 2, (0, 3): 3, (1, 2): 1.5, (1, 3): 2.5, (2, 3): 3.5, (1, 0): 1, (2, 0): 2, (3, 0): 3, (2, 1): 1.5, (3, 1): 2.5, (3, 2): 3.5}  # Travel times
C_v = {1: 100, 2: 100}  # Vehicle capacities
cost_km = 1
speed = 1
t_p = {1: 5, 2: 10}  # Service times
deadlines = {1: (20240610, 18), 2: (20240608, 16), 3: (20240609, 15)}  # Deadlines

# Convert days to time in hours since start of planning horizon (e.g., 08-06-2024 00:00)
start_date = datetime.strptime("2024-06-08", "%Y-%m-%d")
time_window_hours = {}
for c, tws in TW.items():
    time_window_hours[c] = []
    for (day, start_hour, end_hour) in tws:
        day_date = datetime.strptime(str(day), "%Y%m%d")
        delta_days = (day_date - start_date).days
        start_time = delta_days * 24 + start_hour
        end_time = delta_days * 24 + end_hour
        time_window_hours[c].append((start_time, end_time))

# Convert deadlines to hours since start of planning horizon
deadline_hours = {}
for c, (day, hour) in deadlines.items():
    day_date = datetime.strptime(str(day), "%Y%m%d")
    delta_days = (day_date - start_date).days
    deadline_time = delta_days * 24 + hour
    deadline_hours[c] = deadline_time

# Create the LP model
model = pulp.LpProblem("VRP", pulp.LpMinimize)

# Decision variables
x = pulp.LpVariable.dicts("x", ((v, i, j) for v in V for i in C + [0] for j in C + [0] if i != j), cat='Binary')
y = pulp.LpVariable.dicts("y", ((c, p, v) for c in C for p in P for v in V), lowBound=0, cat='Integer')
T_v = pulp.LpVariable.dicts("T_v", ((c, v) for c in C for v in V), lowBound=0, cat='Continuous')
z = pulp.LpVariable.dicts("z", ((c, v, tw_idx) for c in C for v in V for tw_idx in range(len(time_window_hours[c]))), cat='Binary')

# Objective function
model += pulp.lpSum(D[i, j] * cost_km * x[v, i, j] for v in V for i in C + [0] for j in C + [0] if i != j)

# Constraints

# Ensure all demand is met
for c in C:
    for p in P:
        model += pulp.lpSum(y[c, p, v] for v in V) == n[c, p]

# Ensure vehicle capacity constraints
for v in V:
    model += pulp.lpSum(w[p] * y[c, p, v] for c in C for p in P) <= C_v[v]

# Ensure time window constraints
for c in C:
    for v in V:
        for tw_idx, (start, end) in enumerate(time_window_hours[c]):
            model += T_v[c, v] >= start - 1e5 * (1 - z[c, v, tw_idx])
            model += T_v[c, v] <= end + 1e5 * (1 - z[c, v, tw_idx])
        model += pulp.lpSum(z[c, v, tw_idx] for tw_idx in range(len(time_window_hours[c]))) == pulp.lpSum(x[v, i, c] for i in C + [0] if i != c)

# Ensure deadlines are met
for c in C:
    for v in V:
        model += T_v[c, v] + pulp.lpSum(t_p[p] * y[c, p, v] for p in P) <= deadline_hours[c]

# Flow conservation constraints
for v in V:
    for i in C + [0]:
        model += pulp.lpSum(x[v, i, j] for j in C + [0] if j != i) == pulp.lpSum(x[v, j, i] for j in C + [0] if j != i)

# Each vehicle leaves and returns to the warehouse
for v in V:
    model += pulp.lpSum(x[v, 0, j] for j in C) == 1
    model += pulp.lpSum(x[v, j, 0] for j in C) == 1

# Ensure each customer is visited at least once
for c in C:
    model += pulp.lpSum(x[v, i, c] for v in V for i in C + [0] if i != c) >= 1
    model += pulp.lpSum(x[v, c, j] for v in V for j in C + [0] if j != c) >= 1

# Time synchronization constraints
M = 1e5  # Large constant
for v in V:
    for i in C:
        for j in C:
            if i != j:
                model += T_v[j, v] >= T_v[i, v] + T[i, j] + pulp.lpSum(t_p[p] * y[i, p, v] for p in P) - M * (1 - x[v, i, j])

# Solve model
model.solve()

# Print solution
if pulp.LpStatus[model.status] == 'Optimal':
    print("Optimal Solution Found:")
    for v in V:
        print(f"Vehicle {v} Route:")
        route = [0]
        while route[-1] != 0 or len(route) == 1:
            i = route[-1]
            for j in C + [0]:
                if i != j and pulp.value(x[v, i, j]) > 0.5:
                    route.append(j)
                    break
        print(" -> ".join(map(str, route)))
    
    for c in C:
        for p in P:
            for v in V:
                if pulp.value(y[c, p, v]) > 0:
                    print(f"Customer {c}, Product {p}, Vehicle {v}: {int(pulp.value(y[c, p, v]))}")
else:
    print("No optimal solution found")



Optimal Solution Found:
Vehicle 1 Route:
0 -> 3 -> 0
Vehicle 2 Route:
0 -> 2 -> 1 -> 0
Customer 1, Product 1, Vehicle 1: 2
Customer 1, Product 2, Vehicle 2: 1
Customer 2, Product 1, Vehicle 1: 1
Customer 2, Product 2, Vehicle 1: 1
Customer 3, Product 1, Vehicle 2: 3
Customer 3, Product 2, Vehicle 2: 2


In [14]:
import sqlite3

# Connect to SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect('vrp.db')
cursor = conn.cursor()

# Create tables
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Vehicles (
        VehicleID INTEGER PRIMARY KEY,
        Capacity FLOAT
    )
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS Customers (
        CustomerID INTEGER PRIMARY KEY,
        Name TEXT
    )
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS Products (
        ProductID INTEGER PRIMARY KEY,
        Weight FLOAT
    )
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS CustomerDemands (
        CustomerID INTEGER,
        ProductID INTEGER,
        Demand INTEGER,
        PRIMARY KEY (CustomerID, ProductID),
        FOREIGN KEY (CustomerID) REFERENCES Customers(CustomerID),
        FOREIGN KEY (ProductID) REFERENCES Products(ProductID)
    )
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS TimeWindows (
        CustomerID INTEGER,
        Day TEXT,
        StartHour INTEGER,
        EndHour INTEGER,
        PRIMARY KEY (CustomerID, Day, StartHour, EndHour),
        FOREIGN KEY (CustomerID) REFERENCES Customers(CustomerID)
    )
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS Distances (
        FromID INTEGER,
        ToID INTEGER,
        Distance FLOAT,
        TravelTime FLOAT,
        PRIMARY KEY (FromID, ToID),
        FOREIGN KEY (FromID) REFERENCES Locations(LocationID),
        FOREIGN KEY (ToID) REFERENCES Locations(LocationID)
    )
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS Deadlines (
        CustomerID INTEGER PRIMARY KEY,
        Deadline TEXT,
        FOREIGN KEY (CustomerID) REFERENCES Customers(CustomerID)
    )
''')

# Commit changes and close connection
conn.commit()
conn.close()

print("Database 'vrp.db' created successfully.")


Database 'vrp.db' created successfully.
